<img src = "https://miro.medium.com/max/854/0*bfgOZEZqaXYB2yRo.jpg" style = "height:300px;width:100%;">
<h1 style = "float:center;">A Credit Scoring Model</h1>
<p> The Credit score willbe a probability attached to each new customer on their ability to pay a loan based on data from past loans. Those with low probabilities will either receive no loans or just receive very low loan until their ability to pay is analised via experience</p> 
<p> The data comes from the popular peer to peer lending website:  <a href = "https://www.lendingclub.com/info/download-data.action">Lending Club</p>

<h2>Data Exploration and Cleaning</h2>
<h3>Cleaning</h3>

In [7]:
#Import the Necessary Libraries
import os
os.listdir('.ipynb_checkpoints')
import pandas as pd
import os
import time

import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

#Print all rows and columns. Dont hide any
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import pickle

In [6]:
# Load the dataset
%time
loans = pd.read_csv('LoanStats3a.csv')


Wall time: 0 ns


FileNotFoundError: File b'LoanStats3a.csv' does not exist

In [ ]:
#print(loans.columns)
print(loans.shape)
#loans.isnull().sum()
loans = loans.dropna(axis='columns', how='all')
loans = loans.dropna(axis=0, how='all')
print(loans.shape)

thresh = len(loans) * .6
loans.dropna(thresh = thresh, axis = 1, inplace = True)
# Checking for missing values in filtered data frame
loans.apply(lambda x: sum(x.isnull()),axis=0) 
print(loans.shape)
loans.head(2)

In [ ]:
#loans.isnull().sum()
loans = loans[(loans['loan_status']=='Fully Paid') | (loans['loan_status']=='Charged Off')]
loans['loan_status_bin'] = loans['loan_status'].map({'Charged Off': 1, 'Fully Paid': 0}) # 0 good loans 1 bad loans
loans.loan_status_bin.value_counts()

In [ ]:
# The variables used for the analysis
loans.columns
columns  = ['loan_amnt',      # The Amount of Loan Requested
            'loan_status_bin',# A Binary loan status with 1 being bad loans and 0 being Good loans
            'funded_amnt',    #The amount that was Given
            'term',           # 3 or five year loan
            'int_rate',       # the interest rate charged
            'installment',    # The intallment amount paid
            'emp_length',     #How long the person has bein under employment
            'home_ownership', #What ownership they have for their home, rental owned or mortgage
            'annual_inc',     # annual income
            'verification_status',# was the income source verified
           'loan_status',     # is the loan paid fully or defaulted
            'purpose',        # The purpose for taking the loan
           ]
print(len(columns))
loans.term.value_counts()

In [ ]:
# Check for Null values
df = loans[columns]
print(df.shape)
df.isnull().sum()
#df['emp_length'].value_counts()

In [ ]:
#data imputation
#df['emp_length'] = df['emp_length'].fillna('10+ years')
interest =  [float(i[:6]) for i in df.int_rate] # converts the interest rates to numerical value
df.int_rate = interest
df.info()

In [ ]:
# create Dummies for all categorical variables
lis = ['term','emp_length','home_ownership','verification_status','purpose']
df[lis[2]] = ['OTHER_Home_Ownership' if (i == 'OTHER' or i == 'NONE') else i for i in df[lis[2]]] # rename 'other'
df[lis[4]] = ['OTHER_Purposes' if (i == 'other' ) else i for i in df[lis[4]]]# rename 'other'
df['emp_length'] = df['emp_length'].fillna(df['emp_length'].mode()[0])

df_clean = pd.DataFrame(index = df.index)
for i in lis:
    H = df[i]
    
    H_dum = pd.get_dummies(H)
    
    B = pd.concat((df_clean,H_dum),1)
    df_clean = B
    
    print(B.shape)
df_cleans = pd.concat((df,B),1)
df_cleans.isnull().sum()
df_cleans[df_cleans.columns.difference(lis + ['loan_status_bin','loan_status'])].info()
df_cleans.columns

# Data Exploration and Visualisation

In [ ]:
df.hist(figsize=(20,20),color = "green")

In [ ]:

def emp_to_num(term):
    """converts the employment column to numerical values ranging from 0 - 10"""
    if pd.isna(term):
        return None
    elif term[2]=='+':
        return 10
    elif term[0]=='<':
        return 0
    else:
        return int(term[0])

df['emp_length_num'] = df['emp_length'].apply(emp_to_num)
plt.title('Employment in Years')
(df['emp_length_num'].value_counts().sort_index()/len(df)).plot.bar()


In [ ]:
# is there a connection between the employment and the default rate
df.groupby('emp_length_num')['loan_status_bin'].mean().plot.bar()
'''Intrestingly there is no obvious connection between employment length and default rate. For example employment lengths of 8 and 9 years have almost the same default rate as employment lengths of up to 1 year. But what we can see is that an employment length of 0 or 1 has a high default rate and an emloyment length of more than 10 years has a really low default rate. So we are going to transform the feature into two features called 'long_emp' and 'short_emp'.'''

In [ ]:
# distribution of interest rates
#(df['int_rate'].dropna()/len(df)).plot.hist(bins=10)
# Purpose for the loans

(df['purpose'].value_counts()/len(df)).plot.bar()
plt.xticks(rotation= 70)

In [ ]:
# The term of loans vis a vis the default rate
(df['term'].value_counts()/len(df)).plot.bar(title='value counts') # more short term loans than long term loans
df.groupby('term')['loan_status_bin'].mean().plot.bar(title='default rate') # higher default rate in longer loans

In [ ]:
#Those that rent their houses have a high default rate
df[(df['home_ownership']=='MORTGAGE') | (df['home_ownership']=='OWN')| (df['home_ownership']=='RENT')].groupby('home_ownership')['loan_status_bin'].mean().plot.bar(title='default rate')

# Train the Machine Learning Model
### Create an X and Y  - input output dataset

In [ ]:
# Balancing the data ti improve overall performance
filter1 = df_cleans.loan_status_bin == 0
filter2 = df_cleans.loan_status_bin == 1
good_loans = df_cleans[filter1]
bad_loans = df_cleans[filter2]
slices = good_loans.iloc[:5670,:]
new = pd.concat((bad_loans,slices),0)
print(good_loans.shape,bad_loans.shape,new.shape)

In [ ]:
# Import the required libraries for spliting, testing and training the model
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

X = df_cleans[df_cleans.columns.difference(lis + ['loan_status_bin','loan_status'])]
Y = df_cleans['loan_status_bin']

X1 = new[df_cleans.columns.difference(lis + ['loan_status_bin','loan_status'])]
Y1 = new['loan_status_bin']
X11 = X1.iloc[:11339, :]
X11.to_csv("X1.csv",index = False)
Y1.to_csv("Y1.csv", index = False)
print(X.shape,Y.shape)

X_train,X_test,Y_train,Y_test = train_test_split(X1,Y1,test_size = 0.2)
sc = MinMaxScaler()
clf1 = RandomForestClassifier(n_estimators = 100 ,verbose=1,random_state=324) 
clf = LogisticRegression(penalty='l1', C=0.01,verbose = 1)

pipe_lr = Pipeline([('scaler', sc), ('clf1', clf1)])

#Train the model
pipe_lr.fit(X_train, Y_train)
# Save the model to the local machine
filename = 'model.pickle'
pickle.dump(pipe_lr, open(filename, 'wb'))
X1.columns


In [ ]:
# Classification report to see how well the model performs
preds = pipe_lr.predict(X_test)

print(classification_report(Y_test,preds))
print(confusion_matrix(Y_test,preds))

In [ ]:
# Model Evaluation _ a visual approach using the (receiver_operating_curve (ROC))
test_probas = pipe_lr.predict_proba(X_test)[:,1]
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
ax = plt.figure(figsize = (7,7))
fpr, tpr, tresholds = roc_curve(Y_test, test_probas)
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color = 'red')
plt.title('ROC')
plt.xlabel('FPR')
plt.ylabel('TPR')

print('ROC-AUC-score: ', roc_auc_score(Y_test, test_probas))

In [ ]:
# Test using Random Data
filter1 = df_cleans.loan_status_bin ==0
supers = df_cleans[filter1]
row = supers.iloc[0:4,:]
row = row[row.columns.difference(lis + ['loan_status_bin','loan_status'])]
def credit_score(row):
    probability = pipe_lr.predict_proba(row)
    df = pd.DataFrame(probability)
    print(probability[:,0])
    thresh = 10
    return probability[:,0]*thresh



credit_score(row)




    

<h1>Actionable items based on the model</h1>
* The model can be used as is on new customers whose data would be uploaded as csv and the output would be a credit score, the compnay would then use a score card to assign loans based on the credit score.
* The model can also be used by non datascientists using a dashboard created using ipywidgets or equivalent complete with a graphical user interface (GUI)
* The model can be used as the analytical base for a website where users log in and fill in a form, the model would use the data from this form to generate credit score that would be used to give loans. this can be achieved through use of Flask or Django in python or their equivalents in other programing languages. An example of this is vailable on github <a href = "https://github.com/sophicist/Flask-Apps/tree/master/credit%20scorer">  here</a>
* this model can also be interlinked with an already existing website that would approve loans based on the credit scores 

# Ways to improve the models
* use the full lendeable dataset
* use ensemble machine learning models such as voting to improve performance
* use deep neural networks and test performance
* use Bayes statistics to improve the model and to make it more Kenyan ( add more questions on the flask form)
* convert the dollars to Kenyan currency
* add the number of variables from the current 11
* link the model to the crb ratings and Mpesa data to get more information on the client
* Create a baseline model for the same to see how well the model performs
* look for a kenyan dataset and test the 
